# Decentralization Planning

## Objective and Prerequisites

Ready for a mathematical optimization modeling challenge? Put your skills to the test with this example, where you’ll learn how to model and solve a decentralization planning problem. You’ll have to figure out – given a set of departments of a company, and potential cities where these departments can be located – the “best” location for each department in order to maximize gross margins.

This model is example 10 from the fifth edition of Model Building in Mathematical Programming by H. Paul Williams on pages 265 and 317-319.


## Problem Description

A large company wants to move some of its departments out of London. Doing so will result in reduced costs in some areas
(such as cheaper housing, government incentives, easier recruitment, etc.), and increased costs in other areas (such as communication between departments). The cost implications for all possible locations of each department have been calculated.
The goal is to determine where to locate each department in order to maximize the total difference between the reduced costs from relocating and the increased communication costs between departments.

The company comprises five departments (A, B, C, D and E). The possible cities for relocation are Bristol and Brighton, or a department may be kept in London. None of these cities (including London) may be the location for more than three of the departments.


## Model Formulation

### Sets and Indices

$d,d2 \in \text{Departments}=\{A,B,C,D,E\}$

$c,c2 \in \text{Cities}=\{\text{Bristol}, \text{Brighton}, \text{London}\}$

### Parameters

$\text{benefit}_{d,c} \in \mathbb{R}^+$: Benefit -in thousands of dollars per year, derived from relocating department $d$ to city $c$.

$\text{communicationCost}_{d,c,d2,c2} \in \mathbb{R}^+$: Communication cost -in thousands of dollars per year, derived from relocating department $d$ to city $c$ and relocating department $d2$ to city $c2$.

We define the set $dcd2c2 = \{(d,c,d2,c2) \in \text{Departments} \times \text{Cities} \times \text{Departments} \times \text{Cities}: \text{communicationCost}_{d,c,d2,c2} > 0  \}$

### Decision Variables

$\text{locate}_{d,c} \in \{0,1 \}$: This binary variable is equal 1, if department $d$ is located at city $c$, and 0 otherwise.

$y_{d,c,d2,c2} = \text{locate}_{d,c}*\text{locate}_{d2,c2} \in \{0,1 \}$: This auxiliary binary variable is equal 1, if department $d$ is located at city $c$ and department $d2$ is located at city $c2$, and 0 otherwise.

### Constraints

**Department location**: Each department must be located in only one city.

\begin{equation}
\sum_{c \in \text{Cities}} \text{locate}_{d,c} = 1 \quad \forall d \in \text{Departments}
\end{equation}

**Departments limit**: No city may be the location for more than three departments.

\begin{equation}
\sum_{d \in \text{Departments}} \text{locate}_{d,c} \leq 3 \quad \forall c \in \text{Cities}
\end{equation}

**Logical Constraints**:

- If $y_{d,c,d2,c2} = 1$ then $\text{locate}_{d,c} = 1$ and $\text{locate}_{d2,c2} = 1$.

\begin{equation}
y_{d,c,d2,c2} \leq \text{locate}_{d,c} \quad \forall (d,c,d2,c2) \in dcd2c2
\end{equation}

\begin{equation}
y_{d,c,d2,c2} \leq \text{locate}_{d2,c2} \quad \forall (d,c,d2,c2) \in dcd2c2
\end{equation}

- If $\text{locate}_{d,c} = 1$ and $\text{locate}_{d2,c2} = 1 $ then $y_{d,c,d2,c2} = 1$.

\begin{equation}
\text{locate}_{d,c} + \text{locate}_{d2,c2} - y_{d,c,d2,c2} \leq 1 \quad \forall (d,c,d2,c2) \in dcd2c2
\end{equation}

### Objective Function

**Gross margin**: Maximize the gross margin of relocation.

\begin{equation}
\text{Maximize} \quad Z = \sum_{d \in \text{Departments}} \sum_{c \in \text{Cities}} \text{benefit}_{d,c} \cdot \text{locate}_{d,c} -
\sum_{d,c,d2,c2 \in dcd2c2} \text{communicationCost}_{d,c,d2,c2} \cdot y_{d,c,d2,c2}
\end{equation}


## Python Implementation


In [63]:
import pandas as pd
import pyomo.environ as pyo

## Input data

We define all the input data for the model.


In [64]:
# Lists of deparments and cities

Deparments = ["A", "B", "C", "D", "E"]
Cities = ["Bristol", "Brighton", "London"]

# Create a dictionary to capture benefits -in thousands of dollars from relocation.

benefit = {
    ("A", "Bristol"): 10,
    ("A", "Brighton"): 10,
    ("A", "London"): 0,
    ("B", "Bristol"): 15,
    ("B", "Brighton"): 20,
    ("B", "London"): 0,
    ("C", "Bristol"): 10,
    ("C", "Brighton"): 15,
    ("C", "London"): 0,
    ("D", "Bristol"): 20,
    ("D", "Brighton"): 15,
    ("D", "London"): 0,
    ("E", "Bristol"): 5,
    ("E", "Brighton"): 15,
    ("E", "London"): 0,
}
d2c = list(benefit.keys())

# Create a dictionary to capture the communication costs -in thousands of dollars from relocation.

communicationCost = {
    ("A", "London", "C", "Bristol"): 13,
    ("A", "London", "C", "Brighton"): 9,
    ("A", "London", "C", "London"): 10,
    ("A", "London", "D", "Bristol"): 19.5,
    ("A", "London", "D", "Brighton"): 13.5,
    ("A", "London", "D", "London"): 15,
    ("B", "London", "C", "Bristol"): 18.2,
    ("B", "London", "C", "Brighton"): 12.6,
    ("B", "London", "C", "London"): 14,
    ("B", "London", "D", "Bristol"): 15.6,
    ("B", "London", "D", "Brighton"): 10.8,
    ("B", "London", "D", "London"): 12,
    ("C", "London", "E", "Bristol"): 26,
    ("C", "London", "E", "Brighton"): 18,
    ("C", "London", "E", "London"): 20,
    ("D", "London", "E", "Bristol"): 9.1,
    ("D", "London", "E", "Brighton"): 6.3,
    ("D", "London", "E", "London"): 7,
    ("A", "Bristol", "C", "Bristol"): 5,
    ("A", "Bristol", "C", "Brighton"): 14,
    ("A", "Bristol", "C", "London"): 13,
    ("A", "Bristol", "D", "Bristol"): 7.5,
    ("A", "Bristol", "D", "Brighton"): 21,
    ("A", "Bristol", "D", "London"): 19.5,
    ("B", "Bristol", "C", "Bristol"): 7,
    ("B", "Bristol", "C", "Brighton"): 19.6,
    ("B", "Bristol", "C", "London"): 18.2,
    ("B", "Bristol", "D", "Bristol"): 6,
    ("B", "Bristol", "D", "Brighton"): 16.8,
    ("B", "Bristol", "D", "London"): 15.6,
    ("C", "Bristol", "E", "Bristol"): 10,
    ("C", "Bristol", "E", "Brighton"): 28,
    ("C", "Bristol", "E", "London"): 26,
    ("D", "Bristol", "E", "Bristol"): 3.5,
    ("D", "Bristol", "E", "Brighton"): 9.8,
    ("D", "Bristol", "E", "London"): 9.1,
    ("A", "Brighton", "C", "Bristol"): 14,
    ("A", "Brighton", "C", "Brighton"): 5,
    ("A", "Brighton", "C", "London"): 9,
    ("A", "Brighton", "D", "Bristol"): 21,
    ("A", "Brighton", "D", "Brighton"): 7.5,
    ("A", "Brighton", "D", "London"): 13.5,
    ("B", "Brighton", "C", "Bristol"): 19.6,
    ("B", "Brighton", "C", "Brighton"): 7,
    ("B", "Brighton", "C", "London"): 12.6,
    ("B", "Brighton", "D", "Bristol"): 16.8,
    ("B", "Brighton", "D", "Brighton"): 6,
    ("B", "Brighton", "D", "London"): 10.8,
    ("C", "Brighton", "E", "Bristol"): 28,
    ("C", "Brighton", "E", "Brighton"): 10,
    ("C", "Brighton", "E", "London"): 18,
    ("D", "Brighton", "E", "Bristol"): 9.8,
    ("D", "Brighton", "E", "Brighton"): 3.5,
    ("D", "Brighton", "E", "London"): 6.3,
}

dcd2c2 = list(communicationCost.keys())

## Model Deployment

We create a model and the variables. These binary decision variables define the city at which each department will be located.

Solving quadratic assignment problems with Gurobi is as easy as configuring the global parameter `nonConvex`, and setting this parameter to the value of 2.


In [65]:
m = pyo.ConcreteModel("decentralization")

# locate deparment d at city c
m.locate = pyo.Var(d2c, domain=pyo.Binary, name="locate")

# logical variable
m.y = pyo.Var(dcd2c2, domain=pyo.Binary, name="logical")


# Department location constraint
def department_location(m, d):
    return sum(m.locate[d, c] for c in Cities) == 1


m.department_location = pyo.Constraint(Deparments, rule=department_location)


# Limit on number of departments
def departments_limit(m, c):
    return sum(m.locate[d, c] for d in Deparments) <= 3


m.departments_limit = pyo.Constraint(Cities, rule=departments_limit)


# logical constraints
def logical_1(m, d, c, d2, c2):
    return m.y[d, c, d2, c2] <= m.locate[d, c]


m.logical_1 = pyo.Constraint(dcd2c2, rule=logical_1)


def logical_2(m, d, c, d2, c2):
    return m.y[d, c, d2, c2] <= m.locate[d2, c2]


m.logical_2 = pyo.Constraint(dcd2c2, rule=logical_2)


def logical_3(m, d, c, d2, c2):
    return m.locate[d, c] + m.locate[d2, c2] - m.y[d, c, d2, c2] <= 1


m.logical_3 = pyo.Constraint(dcd2c2, rule=logical_3)


def obj(m):
    return sum(benefit[d, c] * m.locate[d, c] for d, c in d2c) - sum(
        communicationCost[d, c, d2, c2] * m.y[d, c, d2, c2] for d, c, d2, c2 in dcd2c2
    )


m.obj = pyo.Objective(rule=obj, sense=pyo.maximize)

In [66]:
# solve
pyo.SolverFactory("cbc").solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 14.9
  Upper bound: 14.9
  Number of objectives: 1
  Number of constraints: 170
  Number of variables: 69
  Number of binary variables: 69
  Number of integer variables: 69
  Number of nonzeros: 64
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.42
  Wallclock time: 0.66
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound:

## Analysis

The optimal relocation plan and associated financial report follows.


In [68]:
relocation_plan = pd.DataFrame(columns=["Department", "City"])

count = 0

for c in Cities:
    for d in Deparments:
        if m.locate[d, c].value > 0.5:
            count += 1
            relocation_plan = pd.concat(
                [
                    relocation_plan,
                    pd.DataFrame.from_dict([{"Department": d, "City": c}]),
                ],
                ignore_index=True,
            )
relocation_plan.index = [""] * count
relocation_plan

,Department,City
,A,Bristol
,D,Bristol
,B,Brighton
,C,Brighton
,E,Brighton


In [69]:
print(
    "\n\n_________________________________________________________________________________"
)
print(f"Financial report")
print(
    "_________________________________________________________________________________"
)
total_benefit = 0
for c in Cities:
    for d in Deparments:
        if m.locate[d, c].value > 0.5:
            total_benefit += 1000 * benefit[d, c]

dollars_benefit = "${:,.2f}".format(total_benefit)
print(f"The yearly total benefit is {dollars_benefit} dollars")

total_communication_cost = 0
for d, c, d2, c2 in dcd2c2:
    if m.locate[d, c].value * m.locate[d2, c2].value > 0.5:
        total_communication_cost += 1000 * communicationCost[d, c, d2, c2]

dollars_communication_cost = "${:,.2f}".format(total_communication_cost)
print(f"The yearly total communication cost is {dollars_communication_cost} dollars")

total_gross_margin = total_benefit - total_communication_cost
dollars_gross_margin = "${:,.2f}".format(total_gross_margin)
print(f"The yearly total gross margin is {dollars_gross_margin} dollars")



_________________________________________________________________________________
Financial report
_________________________________________________________________________________
The yearly total benefit is $80,000.00 dollars
The yearly total communication cost is $65,100.00 dollars
The yearly total gross margin is $14,900.00 dollars


## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.
